## Procesamiento de Lenguaje Natural

![Colegio Bourbaki](./Images/Bourbaki.png)

## Fast Text: Representación de palabras mediante subpalabras

Link de Interés: https://fasttext.cc/

FastText es un método de embedding de palabras desarrollado por **Facebook AI Research (FAIR)**. Es una extensión del modelo Word2Vec, pero introduce una mejora clave: en lugar de aprender vectores solo para palabras completas, representa cada palabra como un conjunto de n-gramas de caracteres.

- Representación mediante n-gramas

Por ejemplo, si tomamos la palabra “artificial” y definimos n=3:

Los n-gramas de caracteres serían:

<ar, art, rti, tif, ifi, fic, ici, cia, ial, al>

(A menudo se añaden los delimitadores de inicio y fin, como < y >, para preservar el contexto).

Así, la representación vectorial de la palabra se obtiene como la suma o promedio de los vectores de sus n-gramas.

- Ventajas sobre Word2Vec

Generalización a palabras desconocidas (OOV):
Si una palabra no se vio durante el entrenamiento, FastText puede descomponerla en sus subpalabras (n-gramas) y construir un vector aproximado, lo que Word2Vec no puede hacer.

Comprensión morfológica:
Al basarse en subpalabras, captura información sobre prefijos, sufijos y raíces, lo que mejora la representación de palabras con similitudes morfológicas (por ejemplo, “correr”, “corriendo”, “corredor”).

- Modelo de entrenamiento

El modelo subyacente es similar a Skip-gram o CBOW (como en Word2Vec), pero cada palabra se trata como la suma de sus n-gramas de caracteres.
Aunque internamente no modela explícitamente la estructura lingüística de la oración, utiliza una ventana deslizante de contexto sobre las palabras, como un modelo de bolsa de palabras (bag of words).

![fastText](./Images/cbow-sg.png)

**Aplicaciones prácticas**

FastText se puede aplicar a numerosos problemas relacionados con el lenguaje natural:

- Corrección ortográfica y autocompletado.

- Sistemas de recomendación y búsqueda: mejores sugerencias de productos o consultas.

- Chatbots y atención al cliente: comprensión de entradas con errores tipográficos.

- Análisis de sentimientos y reseñas.

Se pueden usar conjuntos de datos como:

- Consultas de búsqueda de usuarios.

- Conversaciones de chatbots.

- Reseñas y valoraciones.

Estos modelos pueden mejorar la experiencia del cliente al ofrecer sugerencias más precisas, autocorrección, o resultados más relevantes.


FastText es un modelo de embedding basado en subpalabras (n-gramas) que mejora a Word2Vec al poder representar palabras raras o no vistas y captar mejor la morfología del idioma.
Gracias a esto, es especialmente útil en aplicaciones prácticas donde la entrada del usuario puede ser ruidosa o diversa.

---

### Corrector ortográfico con FastText

En este notebook construimos un **corrector ortográfico inteligente** combinando técnicas clásicas de NLP con **embeddings FastText**.

### Workflow

1. **Tokenización del texto**  
   Dividimos las oraciones en palabras, espacios y signos de puntuación.  
   Solo intentamos corregir las palabras alfabéticas, manteniendo el resto intacto.

2. **Generación de candidatos (edits)**  
   Para cada palabra mal escrita, generamos posibles variantes aplicando operaciones de edición:
   - Eliminación de una letra  
   - Inserción de una letra  
   - Sustitución de una letra  
   - Transposición de dos letras seguidas  

   Además, consideramos los errores más comunes de teclado (QWERTY), donde confundir teclas adyacentes tiene menor costo.

3. **Distancia de edición ponderada**  
   Calculamos qué tan diferente es cada candidato de la palabra original, asignando costos menores a errores típicos de tipeo.

4. **Embeddings de FastText**  
   Usamos vectores de palabras (FastText) para comparar la similitud semántica entre la palabra mal escrita y cada candidato.  
   Esto permite corregir incluso palabras fuera del vocabulario (OOV) gracias a los sub-tokens de FastText.

5. **Ranking de candidatos**  
   Combinamos:
   - Similitud de embeddings  
   - Frecuencia de la palabra en el corpus  
   - Distancia de edición  

   El mejor candidato según esta puntuación se elige como corrección.

6. **Reconstrucción del texto**  
   Reemplazamos solo las palabras corregidas y dejamos intactos los espacios y la puntuación, devolviendo la oración completa.

---

En resumen: el sistema propone correcciones ortográficas basadas no solo en la cercanía de edición, sino también en la **probabilidad de uso real** (frecuencia) y la **cercanía semántica**

### Librerias

In [1]:
# Utilities
from __future__ import annotations
import math
import re
import string
from collections import Counter, defaultdict
from collections.abc import Iterable
from dataclasses import dataclass
import numpy as np

# Wordfreq for word frequencies
from wordfreq import zipf_frequency 

# Gensim for FastText
import gensim.downloader as api
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

# NLTK for corpus handling
import nltk
nltk.download("brown")
from nltk.corpus import brown

[nltk_data] Downloading package brown to /home/pdconte/nltk_data...
[nltk_data]   Package brown is already up-to-date!


### Funciones de ayuda

In [2]:
# Regex for tokenization
TOKEN_RE = re.compile(r"([A-Za-z]+|[^A-Za-z\s]+|\s+)")

Usamos esa regex para separar el texto en tokens manteniendo todo, de manera que al corregir podamos reconstruir la oración sin perder espacios o símbolos.

In [3]:
def tokenize(text: str) -> list[str]:
    """Split into tokens: words, whitespace, punctuation blocks. Keep everything to reconstruct."""
    return TOKEN_RE.findall(text)

In [4]:
def is_word(tok: str) -> bool:
    '''Returns true if token is a word (only letters).'''
    return tok.isalpha()

Importamos todas las letras minúsculas del alfabeto inglés desde la librería estándar string.
Esto se usa para probar reemplazos e inserciones de letras en las palabras.

In [5]:
ALPHABET = string.ascii_lowercase
ALPHABET

'abcdefghijklmnopqrstuvwxyz'

Construimos un diccionario de adyacencia de teclado (layout QWERTY en inglés).

Ejemplo: la letra s tiene como adyacentes a, d, w, x.

Esto sirve para que errores como “s → a” o “s → d” cuesten menos en la distancia de edición (porque son errores de tipeo comunes).

In [6]:
# Simple keyboard adjacency map (US QWERTY). Extend as needed.
KEY_ADJ: dict[str, set[str]] = defaultdict(set)

_adj_rows = [
    "qwertyuiop",
    "asdfghjkl",
    "zxcvbnm",
]
for row in _adj_rows:
    for i, ch in enumerate(row):
        if i > 0:
            KEY_ADJ[ch].add(row[i-1])
        if i < len(row)-1:
            KEY_ADJ[ch].add(row[i+1])

Generamos todas las palabras que están a una edición de distancia de la original:

**deletes** → eliminar una letra.

**transposes** → intercambiar dos letras consecutivas.

**replaces** → reemplazar una letra por otra del alfabeto.

**inserts** → insertar una letra extra en cualquier posición.

Estas son las candidatas para corrección.

In [7]:
def edits(word: str) -> set[str]:
    word = word.lower()
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in ALPHABET]
    inserts  = [L + c + R for L, R in splits for c in ALPHABET]
    return set(deletes + transposes + replaces + inserts)

Implementamos una variante del algoritmo de Damerau-Levenshtein para calcular la distancia entre dos palabras:

Inserción = costo 1

Eliminación = costo 1

Sustitución = costo 1 (pero si la sustitución es entre teclas adyacentes → costo 0.6)

Transposición = costo 0.8

Así modelamos errores más realistas de escritura.

In [8]:
def weighted_edit_distance(src: str, dst: str) -> float:
    """Damerau-Levenshtein style with lower cost for adjacent-key substitutions."""
    src, dst = src.lower(), dst.lower()
    n, m = len(src), len(dst)
    dp = np.zeros((n+1, m+1), dtype=float)
    for i in range(n+1):
        dp[i,0] = i
    for j in range(m+1):
        dp[0,j] = j
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost_sub = 0 if src[i-1]==dst[j-1] else (0.6 if dst[j-1] in KEY_ADJ[src[i-1]] else 1.0)
            dp[i,j] = min(
                dp[i-1,j] + 1,         # deletion
                dp[i, j-1] + 1,        # insertion
                dp[i-1,j-1] + cost_sub # substitution
            )
            if i>1 and j>1 and src[i-1]==dst[j-2] and src[i-2]==dst[j-1]:
                dp[i,j] = min(dp[i,j], dp[i-2,j-2] + 0.8)  # transposition
    return float(dp[n,m])

Link: https://www.geeksforgeeks.org/dsa/damerau-levenshtein-distance/

Vamos a definir un conjunto de parámetros ajustables para balancear los criterios de correción.

Por ejemplo, cuando evaluamos un candidato, el sistema calcula algo por el estilo:

score = sim_w * similitud + freq_w * log_frecuencia - dist_w * distancia

El candidato con mayor score es el que elegimos como corrección.

* sim_w → controla cuánto valoramos la similitud semántica entre la palabra original y el candidato, medida con FastText (cosine similarity).

* freq_w → controla cuánto valoramos que el candidato sea una palabra frecuente en el vocabulario.

* dist_w → controla cuánto penalizamos candidatos que están muy lejos de la palabra original en términos de operaciones de edición.

In [9]:
@dataclass
class RankWeights:
    sim_w: float = 2.0  # peso para la similitud de embeddings (coseno)
    freq_w: float = 1.2  # peso para la frecuencia de la palabra en el corpus
    dist_w: float = 1.0  # penalización por distancia de edición

Vamos a definir la clase principal del corrector ortográfico utilizando FastText. 
Esta clase incluirá métodos para entrenar el modelo, generar candidatos de corrección y seleccionar la mejor corrección basada en una puntuación compuesta, es decir,
la tokenización, la verificación de palabras conocidas, la generación de ediciones, el cálculo de similitud de embeddings y la distancia de edición ponderada, para entrenar, cargar, generar candidatos y corregir texto.

In [ ]:
class SpellChecker:
    def __init__(
        self,
        model=None,  
        vocab: Counter | None = None,  
        weights: RankWeights | None = None, 
    ):
        self.model = model  # modelo FastText (entrenado o cargado)
        self.vocab = vocab or Counter()  # frecuencias de palabras
        self.weights = weights or RankWeights()  # pesos de ranking
        self._wordset: set[str] = (
            set()
        )  # conjunto rápido para saber si una palabra existe
        try:
            zipf_frequency,
            # escala ~0..7 (más alto = más frecuente)
            self.zipf_frequency = zipf_frequency
        except Exception:
            self.zipf_frequency = None

    # Model & vocab
    def train_model(
        self,
        corpus: Iterable[str],
        size: int = 100,
        window: int = 5,
        min_count: int = 1,
        epochs: int = 10,
    ) -> None:
        sentences = corpus
        self.model = FastText(vector_size=size, window=window, min_count=min_count)
        self.model.build_vocab(sentences)
        self.model.train(sentences, total_examples=len(sentences), epochs=epochs)
        self.vocab = Counter(w for sen in sentences for w in sen)
        self._wordset = set(self.vocab)

    def load_pretrained(self, path: str) -> None:
        if path.endswith(".bin"):
            try:
                self.model = KeyedVectors.load_word2vec_format(path, binary=True)
            except Exception:
                self.model = load_facebook_vectors(path)
        else:
            self.model = KeyedVectors.load_word2vec_format(path, binary=False)
        self._wordset = set(self.get_vocab_words())

    def set_vocab_from_corpus(self, corpus: Iterable[str]) -> None:
        words = [w.lower() for s in corpus for w in re.findall(r"[A-Za-z]+", s)]
        self.vocab = Counter(words)
        self._wordset = set(self.vocab)

    def get_vocab_words(self) -> Iterable[str]:
        m = self.model
        if hasattr(m, "key_to_index"):
            return list(m.key_to_index.keys())
        if hasattr(m, "wv") and hasattr(m.wv, "key_to_index"):
            return list(m.wv.key_to_index.keys())
        return list(self.vocab.keys())

    # Scoring
    def _freq_prior(self, w):
        # Prior por frecuencia de palabra:
        # - Si wordfreq: usa Zipf (0..7).
        # - Si no, usa log-frecuencia del vocab manual.
        if self.zipf_frequency is not None:
            return self.zipf_frequency(w.lower(), "en")  # cambiar "en" por otro idioma
        return math.log(self.vocab.get(w.lower(), 0) + 1.0)

    def _cosine(self, a, b):
        # Similitud coseno entre embeddings de palabras
        m = self.model
        if m is None:
            return 0.0
        try:
            if hasattr(m, "wv"):
                va, vb = m.wv[a], m.wv[b]
            else:
                va, vb = m[a], m[b]
            denom = (np.linalg.norm(va) * np.linalg.norm(vb))
            if denom == 0:
                return 0.0
            return float(np.dot(va, vb) / denom)
        except Exception:
            return 0.0

    def score(self, src, cand, prev=None, nxt=None):
        # Componentes
        d   = weighted_edit_distance(src, cand)
        sim = self._cosine(src.lower(), cand.lower())
        fr  = self._freq_prior(cand)

        # Contexto (promedio de similitud con vecinos alfabéticos si existen)
        ctx = 0.0
        cnt = 0
        if prev and prev.isalpha():
            ctx += self._cosine(cand.lower(), prev.lower())
            cnt += 1
        if nxt and nxt.isalpha():
            ctx += self._cosine(cand.lower(), nxt.lower())
            cnt += 1
        if cnt:
            ctx /= cnt

        # Pequeño boost si el candidato ya es igual (evita cambios innecesarios)
        ident_bonus = 0.15 if src.lower() == cand.lower() else 0.0

        # Cálculo final
        return ident_bonus + (2.5 * sim) + (1.4 * fr) + (1.0 * ctx) - (1.2 * d)

    # Candidates
    def known(self, words: Iterable[str]) -> set[str]:
        if self._wordset:
            return {w for w in words if w in self._wordset}
        return {w for w in words if self.vocab.get(w, 0) > 0}

    def candidates(self, word, max_edits=2, k_from_embeddings=200):
        wl = word.lower()
        cands = set()

        # a) vecinos por edición
        cands |= self.known({wl})
        e1 = self.known(edits(wl))
        cands |= e1
        if max_edits >= 2 and e1:
            for w1 in list(e1):  # pueden limitar expansión para no explotar el espacio con [:] en e1
                cands |= self.known(edits(w1))

        # b) vecinos por embeddings (si hay modelo)
        try:
            m = self.model.wv if hasattr(self.model, "wv") else self.model
            if hasattr(m, "most_similar"):
                for w, _ in m.most_similar(wl, topn=k_from_embeddings):
                    cands.add(w)
        except Exception:
            pass
        if not cands:
            cands.add(wl)

        # c) filtro suave por frecuencia (descarta palabras muy raras)
        # umbral ajustable: si existe wordfreq, pedimos Zipf>=2.5; si no, al menos que aparezca en vocab
        filtered = set()
        for w in cands:
            if self.zipf_frequency is not None:
                if self.zipf_frequency(w, "en") >= 2.5:
                    filtered.add(w)
            else:
                if self.vocab.get(w, 0) > 0:
                    filtered.add(w)
        if filtered:
            cands = filtered
        return cands

    # Correction
    def correct_word(self, word, prev=None, nxt=None, max_edits=3):
        if len(word) <= 2:
            return word
        cands = self.candidates(word, max_edits=max_edits)
        best = max(cands, key=lambda c: self.score(word, c, prev=prev, nxt=nxt))
        if word.istitle():
            return best.title()
        if word.isupper():
            return best.upper()
        return best

    def correct(self, text, max_edits=3):
        toks = tokenize(text)
        out = []
        # Para cada token palabra, miramos palabra previa/siguiente (alfabéticas) como contexto
        for i, tok in enumerate(toks):
            if tok.isalpha():
                prev = None
                nxt  = None
                # buscar prev
                j = i - 1
                while j >= 0:
                    if toks[j].isalpha():
                        prev = toks[j]
                        break
                    j -= 1
                # buscar next
                j = i + 1
                while j < len(toks):
                    if toks[j].isalpha():
                        nxt = toks[j]
                        break
                    j += 1
                out.append(self.correct_word(tok, prev=prev, nxt=nxt, max_edits=max_edits))
            else:
                out.append(tok)
        return "".join(out)

* Constructor __init__

Se inicializa el corrector con un modelo de embeddings, un vocabulario y los parámetros de peso.


* Sección “Model & vocab”

train_model → entrena un modelo FastText desde cero sobre un corpus dado.

load_pretrained → carga un modelo ya entrenado desde archivo (.bin o texto).

set_vocab_from_corpus → construye un contador de frecuencias a partir de un corpus.

get_vocab_words → obtiene la lista de palabras conocidas según el modelo o vocabulario.

Esto define la base de conocimiento del corrector.


* Sección “Scoring”

_cosine → calcula la similitud de coseno entre embeddings de dos palabras.

_log_freq → devuelve el logaritmo de la frecuencia de una palabra (más frecuente = más probable).

score → combina: bonus de identidad (si ya coincide la palabra), similitud de embeddings, frecuencia, penalización por distancia de edición.

Da un puntaje total para ordenar candidatos.

* Sección “Candidates”

known → filtra una lista de palabras dejando solo las que están en el vocabulario/modelo.

candidates → genera posibles correcciones:

Variantes por ediciones (borrar, insertar, sustituir, transponer).

Vecinos semánticos según embeddings (most_similar).

Si no encuentra nada, devuelve la palabra original.

Aquí obtenemos qué opciones existen para corregir una palabra.

* Sección “Correction”

correct_word → evalúa todos los candidatos de una palabra y se queda con el de mayor score. Respeta mayúsculas/minúsculas.

correct → tokeniza el texto completo, corrige solo las palabras alfabéticas, y reconstruye el texto final.

Esta es la función que usamos directamente para corregir frases enteras.

Vamos a implementar una función que nos permita construir un corrector. Tenemos 3 modos de uso:

* Modo **pretrained** (pretrained no es None)

Carga un modelo FastText ya entrenado (ej. cc.en.300.bin).
Si el vocabulario está vacío, se inicializa con una lista de hasta 200k palabras del modelo.

* Modo **demo** (demo=True)

Entrena un mini-modelo FastText sobre el TOY_CORPUS.
Esto permite probar el pipeline sin descargar nada.

* Modo **fallback** (ni pretrained ni demo)

Es el modo más básico. Usa solo el vocabulario del TOY_CORPUS, sin embeddings.

In [72]:
corpus = [
    "This is a simple sentence.",
    "Another simple example sentence.",
    "We write code to correct spelling errors.",
    "Fasttext uses subword embeddings.",
    "Spell checking benefits from language models"
]

In [73]:
TOY_CORPUS = [re.findall(r"[A-Za-z]+", s.lower()) for s in corpus]

In [74]:
TOY_CORPUS

[['this', 'is', 'a', 'simple', 'sentence'],
 ['another', 'simple', 'example', 'sentence'],
 ['we', 'write', 'code', 'to', 'correct', 'spelling', 'errors'],
 ['fasttext', 'uses', 'subword', 'embeddings'],
 ['spell', 'checking', 'benefits', 'from', 'language', 'models']]

In [78]:
def build_checker(pretrained=None, demo: bool = True) -> SpellChecker:
    """
    Construye un SpellChecker:
      - Si pretrained es un str -> se asume ruta a archivo .bin/.txt
      - Si pretrained es un objeto gensim (KeyedVectors/FastText) -> se usa directamente
      - Si demo=True -> entrena mini-modelo en TOY_CORPUS
      - Sino -> usa solo vocabulario de TOY_CORPUS (sin embeddings)
    """
    checker = SpellChecker(weights=RankWeights(sim_w=2.0, freq_w=1.2, dist_w=1.0))

    if pretrained is not None:
        # Caso 1: ruta a archivo
        if isinstance(pretrained, str):
            checker.load_pretrained(pretrained)
        else:
            # Caso 2: objeto de modelo gensim ya cargado (ej. api.load)
            checker.model = pretrained

        # inicializar vocabulario si está vacío
        if not checker.vocab:
            words = list(checker.get_vocab_words())[:200000]
            checker.vocab = Counter({w: 1 for w in words})
            checker._wordset = set(words)

    elif demo:
        checker.train_model(TOY_CORPUS, size=300, window=5, min_count=1, epochs=10)

    else:
        checker.vocab = Counter(w for s in TOY_CORPUS for w in s)
        checker._wordset = set(checker.vocab)

    return checker

Veamos cómo usarlo:

In [76]:
sample = "Ths is a smple sentnce with speling erors."

In [79]:
# Build the checker with the vocabulary from TOY_CORPUS (no embeddings)
checker = build_checker(pretrained=None, demo=False)

In [80]:
print("INPUT :", sample)
print("OUTPUT:", checker.correct(sample))

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: This is a simple sentence with spelling errors.


In [81]:
checker.vocab

Counter({'this': 1,
         'is': 1,
         'a': 1,
         'simple': 2,
         'sentence': 2,
         'another': 1,
         'example': 1,
         'we': 1,
         'write': 1,
         'code': 1,
         'to': 1,
         'correct': 1,
         'spelling': 1,
         'errors': 1,
         'fasttext': 1,
         'uses': 1,
         'subword': 1,
         'embeddings': 1,
         'spell': 1,
         'checking': 1,
         'benefits': 1,
         'from': 1,
         'language': 1,
         'models': 1})

In [82]:
checker.candidates('Ths')

{'this'}

In [83]:
word = "Ths"
for cand in checker.candidates(word):
    print(cand, "→", checker.score(word, cand))

this → 8.348


In [84]:
# Build the checker with TOY_CORPUS training
checker = build_checker(pretrained=None, demo=True)
print("INPUT :", sample)
print("OUTPUT:", checker.correct(sample))

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: This is a simple sentence to spelling to.


In [85]:
checker.vocab

Counter({'this': 1,
         'is': 1,
         'a': 1,
         'simple': 2,
         'sentence': 2,
         'another': 1,
         'example': 1,
         'we': 1,
         'write': 1,
         'code': 1,
         'to': 1,
         'correct': 1,
         'spelling': 1,
         'errors': 1,
         'fasttext': 1,
         'uses': 1,
         'subword': 1,
         'embeddings': 1,
         'spell': 1,
         'checking': 1,
         'benefits': 1,
         'from': 1,
         'language': 1,
         'models': 1})

In [86]:
checker.candidates("erors")

{'a',
 'another',
 'benefits',
 'checking',
 'code',
 'correct',
 'errors',
 'example',
 'from',
 'is',
 'language',
 'models',
 'sentence',
 'simple',
 'spell',
 'spelling',
 'this',
 'to',
 'uses',
 'we',
 'write'}

In [87]:
word = "erors"
for cand in checker.candidates(word):
    print(cand, "→", checker.score(word, cand))

to → 6.026423432461916
we → 4.22287869042158
example → 0.8516485245823855
spelling → -2.160801358222961
uses → 2.794191354781389
write → 3.653877890639007
correct → 0.9672654680944976
code → 2.4085649355202907
benefits → 0.9838030936047435
this → 5.304697343029082
language → -2.060023901537061
another → 1.4658790364712475
from → 5.718053382791579
checking → -1.6589563886001706
models → 1.0898189135678118
spell → 0.47752273756265584
simple → 0.21364535671472495
sentence → -1.7862046422362319
a → 4.640576714664698
errors → 5.893901869654655
is → 5.394428218036889


Veamos si utilizamos otro TOY_CORPUS más extenso:

In [88]:
def read_book(url):
    # Make a GET request to the URL
    import requests
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        try:
            # Try to get the text content
            text_content = response.text

            # Check if text_content is None
            if text_content is not None:
                return text_content
            else:
                print("Error: No text content retrieved.")

        except Exception as e:
            # Print an error message if there's an issue with getting the text
            print(f"Error reading content: {e}")
    else:
        # Print an error message if the request was not successful
        print(f"Failed to fetch content. Status code: {response.status_code}")

In [89]:
url = "https://www.gutenberg.org/cache/epub/2413/pg2413.txt"
corpus = read_book(url)


In [90]:
class MyCorpus:
    def __init__(self, corpus_string):
        self.corpus_string = corpus_string
    def __iter__(self):
        for line in self.corpus_string.split("\n"):
            # assume there's one document per line, tokens separated by whitespace
            yield line

In [94]:
corpus = list(MyCorpus(corpus))

In [96]:
corpus

['\ufeffThe Project Gutenberg eBook of Madame Bovary\r',
 '    \r',
 'This ebook is for the use of anyone anywhere in the United States and\r',
 'most other parts of the world at no cost and with almost no restrictions\r',
 'whatsoever. You may copy it, give it away or re-use it under the terms\r',
 'of the Project Gutenberg License included with this ebook or online\r',
 'at www.gutenberg.org. If you are not located in the United States,\r',
 'you will have to check the laws of the country where you are located\r',
 'before using this eBook.\r',
 '\r',
 'Title: Madame Bovary\r',
 '\r',
 'Author: Gustave Flaubert\r',
 '\r',
 'Translator: Eleanor Marx Aveling\r',
 '\r',
 'Release date: February 26, 2006 [eBook #2413]\r',
 '                Most recently updated: April 30, 2025\r',
 '\r',
 'Language: English\r',
 '\r',
 '\r',
 '\r',
 '*** START OF THE PROJECT GUTENBERG EBOOK MADAME BOVARY ***\r',
 '\r',
 '\r',
 '\r',
 '\r',
 'Madame Bovary\r',
 '\r',
 'By Gustave Flaubert\r',
 '\r',
 'Tra

In [97]:
TOY_CORPUS = [re.findall(r"[A-Za-z]+", s.lower()) for s in corpus]

In [98]:
TOY_CORPUS

[['the', 'project', 'gutenberg', 'ebook', 'of', 'madame', 'bovary'],
 [],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and'],
 ['most',
  'other',
  'parts',
  'of',
  'the',
  'world',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions'],
 ['whatsoever',
  'you',
  'may',
  'copy',
  'it',
  'give',
  'it',
  'away',
  'or',
  're',
  'use',
  'it',
  'under',
  'the',
  'terms'],
 ['of',
  'the',
  'project',
  'gutenberg',
  'license',
  'included',
  'with',
  'this',
  'ebook',
  'or',
  'online'],
 ['at',
  'www',
  'gutenberg',
  'org',
  'if',
  'you',
  'are',
  'not',
  'located',
  'in',
  'the',
  'united',
  'states'],
 ['you',
  'will',
  'have',
  'to',
  'check',
  'the',
  'laws',
  'of',
  'the',
  'country',
  'where',
  'you',
  'are',
  'located'],
 ['before', 'using', 'this', 'ebook'],
 [],
 ['title', 'madame', 'bovary'],
 [],
 ['author', 'gustave

In [99]:
checker = build_checker(pretrained=None, demo=True)
print("INPUT :", sample)
print("OUTPUT:", checker.correct(sample))

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: The is a simple sentence with feeling errors.


In [100]:
checker.vocab

Counter({'the': 8162,
         'project': 89,
         'gutenberg': 97,
         'ebook': 13,
         'of': 3717,
         'madame': 245,
         'bovary': 225,
         'this': 447,
         'is': 465,
         'for': 875,
         'use': 34,
         'anyone': 21,
         'anywhere': 3,
         'in': 2097,
         'united': 17,
         'states': 19,
         'and': 3084,
         'most': 45,
         'other': 185,
         'parts': 11,
         'world': 31,
         'at': 1128,
         'no': 341,
         'cost': 8,
         'with': 1218,
         'almost': 50,
         'restrictions': 2,
         'whatsoever': 2,
         'you': 827,
         'may': 33,
         'copy': 13,
         'it': 1191,
         'give': 60,
         'away': 118,
         'or': 308,
         're': 27,
         'under': 101,
         'terms': 28,
         'license': 18,
         'included': 4,
         'online': 4,
         'www': 9,
         'org': 9,
         'if': 270,
         'are': 247,
         '

In [101]:
'spelling' in checker.candidates("speling")

True

In [102]:
print('spelling score:', checker.score('speling', 'spelling'))
print('feeling score:', checker.score('speling', 'feeling'))

spelling score: 6.899865293502807
feeling score: 7.281460279941557


Ahora, vamos a importar un modelo FastText preentrenado desde Gensim y usarlo para construir el corrector ortográfico.


In [103]:
model = api.load("fasttext-wiki-news-subwords-300")

In [104]:
checker = build_checker(pretrained=model, demo=False)
print("INPUT :", sample)
print("OUTPUT:", checker.correct(sample))

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: The is a simple sentence with spelling errors.


In [105]:
checker.vocab

Counter({',': 1,
         'the': 1,
         '.': 1,
         'and': 1,
         'of': 1,
         'to': 1,
         'in': 1,
         'a': 1,
         '"': 1,
         ':': 1,
         ')': 1,
         'that': 1,
         '(': 1,
         'is': 1,
         'for': 1,
         'on': 1,
         '*': 1,
         'with': 1,
         'as': 1,
         'it': 1,
         'The': 1,
         'or': 1,
         'was': 1,
         "'": 1,
         "'s": 1,
         'by': 1,
         'from': 1,
         'at': 1,
         'I': 1,
         'this': 1,
         'you': 1,
         '/': 1,
         'are': 1,
         '=': 1,
         'not': 1,
         '-': 1,
         'have': 1,
         '?': 1,
         'be': 1,
         'which': 1,
         ';': 1,
         'all': 1,
         'his': 1,
         'has': 1,
         'one': 1,
         'their': 1,
         'about': 1,
         'but': 1,
         'an': 1,
         '|': 1,
         'said': 1,
         'more': 1,
         'page': 1,
         'he': 1,
      

In [106]:
checker.candidates('Ths')

{'-its',
 '-this',
 '.its',
 'THats',
 'aas',
 'abs',
 'ads',
 'ah',
 'aha',
 'ahh',
 'ahs',
 'ais',
 'als',
 'ans',
 'aos',
 'ars',
 'as',
 'ash',
 'ass',
 'ath',
 'aus',
 'bas',
 'becuase',
 'bes',
 'bh',
 'bha',
 'bhi',
 'bis',
 'bs',
 'bus',
 'cas',
 'ces',
 'ch',
 'cha',
 'che',
 'chi',
 'cho',
 'chu',
 'cis',
 'cs',
 'cus',
 'cvs',
 'das',
 'des',
 'dh',
 'dha',
 'dis',
 'dis.',
 'ds',
 'eh',
 'eis',
 'es',
 'eth',
 'ethos',
 'everyones',
 'fas',
 'fis',
 'fo',
 'fs',
 'gas',
 'gh',
 'gis',
 'gs',
 'gus',
 'h',
 'ha',
 'happend',
 'has',
 'hb',
 'hc',
 'hd',
 'he',
 'heres',
 'hes',
 'hf',
 'hh',
 'hi',
 'his',
 'hk',
 'hl',
 'hm',
 'hn',
 'ho',
 'hos',
 'hp',
 'hq',
 'hr',
 'hrs',
 'hs',
 'ht',
 'hu',
 'hus',
 'hv',
 'hw',
 'hy',
 'hz',
 'ies',
 'ih',
 'iis',
 'iot',
 'is',
 'ith',
 'itis',
 'its',
 'js',
 'jus',
 'kh',
 'ks',
 'las',
 'les',
 'lis',
 'ls',
 'mans',
 'mas',
 'mes',
 'mh',
 'mis',
 'ms',
 'mts',
 'mus',
 'nas',
 'nes',
 'nh',
 'nhl',
 'nhs',
 'nos',
 'ns',
 'nth'

In [107]:
print("this score:", checker.score("ths", "this"))
print("the score:", checker.score("ths", "the"))

this score: 9.745011578083039
the score: 10.892603388547897


Veamos ahora con el corpus de Brown de NLTK:

In [121]:
checker = build_checker(demo=False)
checker.train_model(list(brown.sents()))
print(checker.correct(sample))

The is a simple Sentence with Feeling Errors.


In [122]:
print("Tamaño de vocabulario:", len(checker.vocab))
print("Palabras más frecuentes:", checker.vocab.most_common(10))

Tamaño de vocabulario: 56057
Palabras más frecuentes: [('the', 62713), (',', 58334), ('.', 49346), ('of', 36080), ('and', 27915), ('to', 25732), ('a', 21881), ('in', 19536), ('that', 10237), ('is', 10011)]


In [123]:
examples = [
    "Ths is a smple sentnce with speling erors.",
    "Shee liks to read boks in the librery.",
    "We are lernig how to corect mispelled wrds.",
    "The quik borwn fox jmps ovr the lazi dog.",
    "Natrual langauge procesing is intersting.",
]

for s in examples:
    print("INPUT :", s)
    print("OUTPUT:", checker.correct(s))
    print("-" * 60)

INPUT : Ths is a smple sentnce with speling erors.
OUTPUT: The is a simple Sentence with Feeling Errors.
------------------------------------------------------------
INPUT : Shee liks to read boks in the librery.
OUTPUT: She like to read books in The library.
------------------------------------------------------------
INPUT : We are lernig how to corect mispelled wrds.
OUTPUT: We Are Large how to correct spelled words.
------------------------------------------------------------
INPUT : The quik borwn fox jmps ovr the lazi dog.
OUTPUT: The quick born for jumps over The la dog.
------------------------------------------------------------
INPUT : Natrual langauge procesing is intersting.
OUTPUT: Natural language Processing is interesting.
------------------------------------------------------------


In [113]:
checker.correct_word("mispelled")


'spelled'

In [114]:
cands = checker.candidates("mispelled")
print("Candidatos para 'speling':", sorted(list(cands)))

Candidatos para 'speling': ['Emptied', 'Queried', 'Siegfried', 'Squeezed', 'Troubled', 'blinded', 'bodied', 'boomed', 'braided', 'briefed', 'bugged', 'buried', 'buzzed', 'calmed', 'charred', 'chilled', 'chuckled', 'copied', 'craved', 'crazed', 'creamed', 'cried', 'crippled', 'crumpled', 'cursed', 'damaged', 'dispelled', 'dodged', 'domiciled', 'doused', 'dreaded', 'drugged', 'echoed', 'embattled', 'envied', 'fear-filled', 'felled', 'ferried', 'frenzied', 'fried', 'fulfilled', 'full-banded', 'full-bodied', 'funneled', 'gobbled', 'golden-crusted', 'gouged', 'gritty-eyed', 'honoured', 'horrified', 'hurried', 'husky-voiced', 'hustled', 'judged', 'killed', 'labeled', 'labelled', 'lied', 'lobbied', 'lodged', 'logged', 'loose-jointed', 'loosened', 'misjudged', 'misled', 'murmured', 'muttered', 'naked', 'nestled', 'nudged', 'occupied', 'padded', 'parodied', 'peaked', 'pitied', 'plagued', 'pleaded', 'pledged', 'plied', 'plugged', 'puffed', 'purged', 'puzzled', 'quadrupled', 'queried', 'rebelled'

In [117]:
word = "mispelled"
for cand in checker.candidates(word):
    print(cand, "→", checker.score(word, cand))

skidded → -1.2574164972305297
craved → -1.4281620836257929
spiced → 0.6648461966514585
full-bodied → -3.01630260157585
tattooed → -0.7135187125205986
scribbled → -0.8846837327480319
horrified → -0.6847704641818995
cried → 0.4083379111289984
labelled → 3.2125119905471795
misled → 3.4350082738399506
chuckled → -0.018614631891250788
red-bellied → -0.49679809904098526
embattled → -0.40052708077430754
queried → -0.4389283623695368
calmed → 0.2789796772003168
boomed → -1.1446942040920263
naked → 1.6724479360580427
envied → -2.1492963938713086
rebuked → -1.5574741647243497
pleaded → 0.6212496116161343
saddled → 0.551214668512344
spelled → 5.384379011154175
misjudged → 1.5253659694194797
golden-crusted → -6.710702860355377
Queried → -0.4389283623695368
echoed → -0.72855621933937
skilled → 3.420541948795319
unskilled → 1.914515377283096
ridiculed → 1.3812791905403135
parodied → -1.9452101974487297
tripled → 0.7711880156993862
reeled → 0.08878745222091666
damaged → 0.6551171133518219
killed → 4.

Ejercicios:

**Conceptos básicos de FastText**

¿Qué es FastText?

¿En qué se diferencia FastText de Word2Vec?

¿Qué tipo de modelo usa internamente FastText?

¿Qué ventaja tiene usar FastText para un corrector ortográfico?

¿Qué limitación tiene FastText frente a modelos como BERT?

**Uso práctico**

¿Qué formato de datos necesita FastText para entrenar un modelo?

¿Qué significan los parámetros vector_size, window y min_count?

¿Qué función se usa en Gensim para cargar un modelo preentrenado de FastText?

¿Cuál es la diferencia entre entrenar un modelo desde cero y cargar uno preentrenado?

¿Por qué es importante que el corpus sea iterable y no un generador puro?

**Clase Spell Checker**

¿Qué papel cumple la distancia de edición en el corrector?

¿Por qué se pondera la distancia de edición según el teclado QWERTY?

¿Qué representa la función score() dentro del SpellChecker?

¿Qué factores se combinan en el cálculo del score()?

¿Qué aporta el contexto (prev, nxt) en la corrección de palabras?

**Corpus y entrenamiento**

¿Qué diferencia hay entre usar un toy corpus y el modelo cc.en.300.bin?

¿Qué tipo de estructura de datos espera FastText en build_vocab() y train()?

¿Qué sucede si el corpus no puede recorrerse más de una vez?

¿Por qué es importante limpiar y tokenizar el texto antes del entrenamiento?

**Extensiones y mejoras**

¿Cómo podrías mejorar la precisión del corrector ortográfico basado en FastText?

¿Qué ocurre si el embedding de una palabra no existe en el modelo?

¿Qué ventajas tiene FastText frente a Word2Vec en palabras inventadas u OOV?

¿Cómo influye el tamaño del vector (vector_size) en el rendimiento y la calidad de las correcciones?

### Referencias

- Word Representations: https://fasttext.cc/docs/en/unsupervised-tutorial.html

- FastText GitHub: https://github.com/facebookresearch/fastText/

- Building a spelling correction/word suggestion module: https://github.com/Svantevith/spelling-correction-module-using-fasttext/blob/master/Spelling_corrector_word_suggestion_module_using_fastText.ipynb

- system_transcription_error_detection_and_correction: https://github.com/kmariael/system_transcription_error_detection_and_correction

- Misspelling-Correction-with-fastText-sequence-labeling: https://github.com/leechehao/Misspelling-Correction-with-fastText-sequence-labeling

- Enriching Word Vectors with Subword Information: https://arxiv.org/pdf/1607.04606

- Bag of Tricks for Efficient Text Classification: https://arxiv.org/abs/1607.01759

- FastText.zip: Compressing text classification models: https://arxiv.org/abs/1612.03651

- Misspelling Oblivious Word Embeddings: https://arxiv.org/pdf/1905.09755

- Zipf’s word frequency law in natural language: A critical review and future directions: https://pmc.ncbi.nlm.nih.gov/articles/PMC4176592/